In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lit, current_timestamp

storageAccountName = "datalakea04fb344bd6a3620"
sas_token = "-"

spark = SparkSession.builder.appName("LeituraDelta").getOrCreate()

spark.conf.set(
    f"fs.azure.sas.silver.{storageAccountName}.blob.core.windows.net", 
    sas_token
)

def salvar_silver(df_silver, tabela): 
    df_silver.write \
        .format("delta") \
        .mode("overwrite") \
        .save(f"/mnt/{storageAccountName}/silver/{tabela}")

def renomear_colunas(df, tabela):
    for coluna in df.columns:
        col_format = coluna.upper()
        col_format = col_format.replace("ID_", "CODIGO_")
        col_format = col_format.replace("DT_", "DATA_")

        df = df.withColumnRenamed(coluna, col_format)
    df = df.drop("DATA_INSERT_BRONZE")
    df = df.drop("FILENAME")
    df = df.withColumn("FILENAME_BRONZE", lit(tabela))
    df = df.withColumn("DATA_INSERT_SILVER", current_timestamp())
    return df

def transformar_corretor():
    df_corretor = spark.read.format("delta").load(f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/corretor")

    df_corretor = df_corretor.withColumn("telefone_corretor", regexp_replace("telefone_corretor", "[^0-9]", ""))
    df_corretor = df_corretor.withColumn("telefone_corretor", col("telefone_corretor").substr(-10, 10))

    df = renomear_colunas(df_corretor, "corretor")

    salvar_silver(df, "corretor")

def transformar(tabela):
    df_bronze = spark.read.format("delta").load(f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/{tabela}")
    
    df = renomear_colunas(df_bronze, tabela)
    
    salvar_silver(df, tabela)
    

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LeituraDelta").getOrCreate()

storageAccountName = "datalakea04fb344bd6a3620"
sas_token = "-"

spark.conf.set(
    f"fs.azure.sas.bronze.{storageAccountName}.blob.core.windows.net", 
    sas_token
)

tables = ["cliente", "imovel", "cobertura", "apolice_seguro", "corretor", "seguradora"]

def transformar(table_name):
    bronze_path = f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/{table_name}"
    df = spark.read.format("delta").load(bronze_path)
    
    silver_path = f"wasbs://silver@{storageAccountName}.blob.core.windows.net/{table_name}"
    df.write.format("delta").mode("overwrite").save(silver_path)

def transformar_corretor():
    table_name = "corretor"
    bronze_path = f"wasbs://bronze@{storageAccountName}.blob.core.windows.net/{table_name}"
    df = spark.read.format("delta").load(bronze_path)
    
    silver_path = f"wasbs://silver@{storageAccountName}.blob.core.windows.net/{table_name}"
    df.write.format("delta").mode("overwrite").save(silver_path)

for table in tables:
    if table == "corretor":
        transformar_corretor()
    else:
        transformar(table)
    
    print(f"Transformação da tabela: {table} finalizada.")



Transformação da tabela: cliente finalizada.
Transformação da tabela: imovel finalizada.
Transformação da tabela: cobertura finalizada.
Transformação da tabela: apolice_seguro finalizada.
Transformação da tabela: corretor finalizada.
Transformação da tabela: seguradora finalizada.


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LeituraDelta").getOrCreate()

storageAccountName = "datalakea04fb344bd6a3620"
sas_token = "-"

spark.conf.set(
    f"fs.azure.sas.silver.{storageAccountName}.blob.core.windows.net",
    sas_token
)

table_name = "corretor"
silver_path = f"wasbs://silver@{storageAccountName}.blob.core.windows.net/{table_name}"

df = spark.read.format("delta").load(silver_path)

df.show()


+-----------+-----------------+---------------+-------------+--------------------+--------------------+--------------------+--------+
|corretor_id|    nome_corretor|numero_registro|seguradora_id|   telefone_corretor|      email_corretor|    dt_insert_bronze|filename|
+-----------+-----------------+---------------+-------------+--------------------+--------------------+--------------------+--------+
|          1| Kimberly Morales|          33974|            1|    001-927-393-4972|lwilliams@example...|2024-12-02 00:04:...|corretor|
|          2|  Martin Robinson|          19663|            2|  886-467-5470x33309|bakernicholas@exa...|2024-12-02 00:04:...|corretor|
|          3|   Kathleen Brown|           7971|            3|+1-775-343-5737x6...|kathleenruiz@exam...|2024-12-02 00:04:...|corretor|
|          4|  Allison Barnett|          94946|            4|       (998)852-0836|robertcrawford@ex...|2024-12-02 00:04:...|corretor|
|          5|   Michael Ritter|          11991|            5| 